# Modeling

In [1]:
import numpy as np
import pandas as pd

from pprint import pprint

import unicodedata

import re

import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

import acquire as ac
import prepare as pr
import preprocessing as pp
import evaluate as ev

---
## Wrangle

In [2]:
df = pr.wrangle_readme_data()
df

,repo,language,readme_contents,clean_readme_contents,len_of_clean_readme_contents
0,rdpeng/ProgrammingAssignment2,R,### Introduction\n\nThis second programming as...,introduction second programming assignment req...,332
1,octocat/Spoon-Knife,HTML,### Well hello there!\n\nThis repository is me...,well hello repository meant provide example fo...,62
2,tensorflow/tensorflow,C++,"<div align=""center"">\n <img src=""https://www....",div aligncenter srchttpswwwtensorfloworgimages...,434
3,SmartThingsCommunity/SmartThingsPublic,Groovy,# SmartThings Public GitHub Repo\n\nAn officia...,smartthings public github repo official list s...,32
4,twbs/bootstrap,JavaScript,"<p align=""center"">\n <a href=""https://getboot...",p aligncenter hrefhttpsgetbootstrapcom srchttp...,567
...,...,...,...,...,...
209,akveo/ngx-admin,TypeScript,"# ngx-admin [<img src=""https://i.imgur.com/oMc...",ngxadmin srchttpsiimgurcomomcxwz0png alteva de...,300
210,swirldev/swirl_courses,R,# swirl courses\n\nThis is a collection of int...,swirl course collection interactive course use...,393
211,jrowberg/i2cdevlib,C++,Jennic platform added!\n\n====================...,jennic platform added note detail project plea...,250
212,etcd-io/etcd,Go,# etcd\n\n[![Go Report Card](https://goreportc...,etcd go report cardhttpsgoreportcardcombadgegi...,630


In [3]:
df.columns

Index(['repo', 'language', 'readme_contents', 'clean_readme_contents',
       'len_of_clean_readme_contents'],
      dtype='object')

In [4]:
df.repo.value_counts().head(18)

crossoverJie/JCSprout                                                          2
tensorflow/models                                                              2
soimort/you-get                                                                2
tensorflow/tensorflow                                                          2
rust-lang/rust                                                                 2
SmartThingsCommunity/SmartThingsPublic                                         2
phonegap/phonegap-start                                                        2
woocommerce/woocommerce                                                        2
axios/axios                                                                    2
octocat/Spoon-Knife                                                            2
apache/incubator-mxnet                                                         2
rdpeng/ProgrammingAssignment2                                                  2
arduino/Arduino             

In [5]:
df.repo.value_counts().tail()

discourse/discourse          1
grafana/grafana              1
elastic/elasticsearch        1
scikit-learn/scikit-learn    1
mdn/learning-area            1
Name: repo, dtype: int64

In [6]:
len(df.repo.unique())

197

In [7]:
# stratification base on language using train_test_split won't work unless we have more than one observation
# per language
df.language.value_counts()

JavaScript          45
Java                35
Python              29
C++                 18
HTML                16
PHP                  9
Jupyter Notebook     9
Go                   9
TypeScript           7
Ruby                 7
CSS                  6
C                    5
Rust                 3
Vue                  3
R                    3
PowerShell           2
C#                   2
Kotlin               2
Groovy               2
Shell                2
Name: language, dtype: int64

---
## Model

---
### Outlier Detour

In [8]:
# lemmas
# list_of_readmes = df.clean_readme_contents.tolist()
# list_of_readmes

In [9]:
# for index in range(len(list_of_readmes)):
#     list_of_readmes[index] = list_of_readmes[index].split()
    

# print(list_of_readmes)

In [10]:
# list_of_list_of_lemmas = [lemma.split() for lemmas in list_of_lemmas]
# list_of_list_of_lemmas

In [11]:
# len(list_of_readmes[0])

In [12]:
# len(list_of_readmes[-1])

In [13]:
# len_of_readmes = [len(readme) for readme in list_of_readmes]
# len_of_readmes

In [14]:
# len_of_readmes.sort()

In [15]:
# print(len_of_readmes)

**Cutoff for length of words in readme = 10**

In [16]:
index_three_list = df[df.index == 3].clean_readme_contents.values.tolist()
index_three_list

['smartthings public github repo official list smartapps device type smartthings link help get started coding right away githubspecific documentationhttpdocssmartthingscomenlatesttoolsandidegithubintegrationhtml full documentationhttpdocssmartthingscom ide simulatorhttpidesmartthingscom community forumshttpcommunitysmartthingscom follow u web twitter httptwittercomsmartthingsdev facebook httpfacebookcomsmartthingsdevelopers']

In [17]:
df[df.index == 3].len_of_clean_readme_contents

3    32
Name: len_of_clean_readme_contents, dtype: int64

**len_of_clean_readme_contents seems to be working**

In [18]:
df.len_of_clean_readme_contents.value_counts().sort_index()

10       1
13       2
14       1
30       2
32       2
        ..
5682     1
7278     1
8039     1
10293    1
16635    1
Name: len_of_clean_readme_contents, Length: 179, dtype: int64

In [19]:
df.shape

(214, 5)

**225 observations before outlier removal; should have 219 after**

**CHECK**

---
### Back on Model Track

#### Train/Test Split

In [20]:
df = pr.wrangle_readme_data()
print(df.shape)
df.head()

(214, 5)


,repo,language,readme_contents,clean_readme_contents,len_of_clean_readme_contents
0,rdpeng/ProgrammingAssignment2,R,### Introduction\n\nThis second programming as...,introduction second programming assignment req...,332
1,octocat/Spoon-Knife,HTML,### Well hello there!\n\nThis repository is me...,well hello repository meant provide example fo...,62
2,tensorflow/tensorflow,C++,"<div align=""center"">\n <img src=""https://www....",div aligncenter srchttpswwwtensorfloworgimages...,434
3,SmartThingsCommunity/SmartThingsPublic,Groovy,# SmartThings Public GitHub Repo\n\nAn officia...,smartthings public github repo official list s...,32
4,twbs/bootstrap,JavaScript,"<p align=""center"">\n <a href=""https://getboot...",p aligncenter hrefhttpsgetbootstrapcom srchttp...,567


## Modeling

Transform your documents into a form that can be used in a machine learning model. You should use the programming language of the repository as the label to predict.

Try fitting several different models and using several different representations of the text (e.g. a simple bag of words, then also the TF-IDF values for each).

Build a function that will take in the text of a README file, and tries to predict the programming language.

**CountVectorizer**

In [21]:
# create cv object
cv = CountVectorizer()

In [22]:
# fit and use the cv object
cv_bag_of_words = cv.fit_transform(df.clean_readme_contents)

In [23]:
cv_bag_of_words

<214x35012 sparse matrix of type '<class 'numpy.int64'>'
	with 76820 stored elements in Compressed Sparse Row format>

**TF-IDF**

In [24]:
# create tfidf vectorizer object
tfidf = TfidfVectorizer()

In [25]:
tfidf_bag_of_words = tfidf.fit_transform(df.clean_readme_contents)

In [26]:
tfidf_bag_of_words

<214x35012 sparse matrix of type '<class 'numpy.float64'>'
	with 76820 stored elements in Compressed Sparse Row format>

**CountVectorizer Bag of Bigrams**

In [27]:
# create cv for bigrams
cv_bigrams = CountVectorizer(ngram_range=(2, 2))

In [28]:
cv_bag_of_bigrams = cv_bigrams.fit_transform(df.clean_readme_contents)

In [29]:
cv_bag_of_bigrams

<214x116381 sparse matrix of type '<class 'numpy.int64'>'
	with 136876 stored elements in Compressed Sparse Row format>

**TF-IDF Bag of Bigrams**

In [30]:
tfidf_bigrams = TfidfVectorizer(ngram_range=(2, 2))

In [31]:
tfidf_bag_of_bigrams = tfidf_bigrams.fit_transform(df.clean_readme_contents)

In [32]:
tfidf_bag_of_bigrams

<214x116381 sparse matrix of type '<class 'numpy.float64'>'
	with 136876 stored elements in Compressed Sparse Row format>

___

In [33]:
# establish features and target for CountVectorizer model
X = cv.fit_transform(df.clean_readme_contents)
y = df.language

In [34]:
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

In [35]:
print(X_train.shape)
print(y_train.shape)
print(X_train.shape[0]/df.shape[0])
print(y_train.shape[0]/df.shape[0])

(171, 35012)
(171,)
0.7990654205607477
0.7990654205607477


In [36]:
print(X_test.shape)
print(y_test.shape)
print(X_test.shape[0]/df.shape[0])
print(y_test.shape[0]/df.shape[0])

(43, 35012)
(43,)
0.20093457943925233
0.20093457943925233


In [37]:
pd.DataFrame(X_train[:5, :].todense(), columns=cv.get_feature_names())

,000,0000,00000,00042,001,002,003,006,007231863956341798,00d2d2,...,zxingbased,zxingcpphttpsgithubcomglassechidnazxingcpp,zxingcpprbhttpsgithubcomglassechidnazxingcpprb,zxingjslibraryhttpsgithubcomzxingjslibrary,zxingnet,zxingobjchttpsgithubcomthelevelupzxingobjc,zxingorg,zynga,zypper,zyszyshttpsgithubcomzyszys
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Model #1: Standard CV**

In [38]:
# create evaluation DataFrame
evaluation = pd.DataFrame(columns=["model_type", "accuracy"])

In [39]:
# create model object
tree = DecisionTreeClassifier(max_depth=5, random_state=56)

# fit model object
tree.fit(X_train, y_train)

# accuracy score
print(f"CV model accuracy score = {tree.score(X_train, y_train):.3}")

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="CV Gini, 5", model_object=tree, X=X_train, y=y_train)

CV model accuracy score = 0.626


**Model #2: TF-IDF**

In [40]:
# establish features and target for tfidf model

# create tfidf vectorizer object
tfidf = TfidfVectorizer()

# use tfidf object to create model features
X = tfidf.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

# create model object
tree = DecisionTreeClassifier(max_depth=5, random_state=56)

# fit model object
tree.fit(X_train, y_train)

# accuracy score
print(f"TF-IDF model accuracy score = {tree.score(X_train, y_train):.3}")

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="TF-IDF Gini, 5", model_object=tree, X=X_train, y=y_train)

TF-IDF model accuracy score = 0.602


**Model #3: CV Bigrams**

In [41]:
# establish features and target for cv model

# create cv_bigrams vectorizer object
cv_bigrams = CountVectorizer(ngram_range=(2, 2))

# use tfidf object to create model features
X = cv_bigrams.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

# create model object
tree = DecisionTreeClassifier(max_depth=5, random_state=56)

# fit model object
tree.fit(X_train, y_train)

# accuracy score
print(f"CV Bigrams model accuracy score = {tree.score(X_train, y_train):.3}")

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="CV Bigrams Gini, 5", model_object=tree, X=X_train, y=y_train)

CV Bigrams model accuracy score = 0.386


**Model #4: TF-IDF Bigrams**

In [42]:
# establish features and target for tfidf model

# create cv_bigrams vectorizer object
tfidf_bigrams = TfidfVectorizer(ngram_range=(2, 2))

# use tfidf object to create model features
X = tfidf_bigrams.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

# create model object
tree = DecisionTreeClassifier(max_depth=5, random_state=56)

# fit model object
tree.fit(X_train, y_train)

# accuracy score
print(f"TF-IDF Bigrams model accuracy score = {tree.score(X_train, y_train):.3}")

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="TF-IDF Bigrams Gini, 5", model_object=tree, X=X_train, y=y_train)

TF-IDF Bigrams model accuracy score = 0.386


**Model #5: TF-IDF Unigrams & Bigrams**

In [43]:
# establish features and target for tfidf model

# create cv_bigrams vectorizer object
tfidf_unigrams_and_bigrams = TfidfVectorizer(ngram_range=(1, 2))

# use tfidf object to create model features
X = tfidf_unigrams_and_bigrams.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

# create model object
tree = DecisionTreeClassifier(max_depth=5, random_state=56)

# fit model object
tree.fit(X_train, y_train)

# accuracy score
print(f"TF-IDF Unigrams & Bigrams model accuracy score = {tree.score(X_train, y_train):.3}")

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="TF-IDF Unigrams & Bigrams Gini, 5", model_object=tree, X=X_train, y=y_train)

TF-IDF Unigrams & Bigrams model accuracy score = 0.591


---

**Create a function to append evaluation metrics (accuracy) to evaluation dataframe**


**CHECK**

---

**Model #6: TF-IDF Logistic Regression**

In [44]:
# establish features and target for tfidf logit model

# create cv_bigrams vectorizer object
tfidf = TfidfVectorizer()

# use tfidf object to create model features
X = tfidf.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

# create model object
logit = LogisticRegression(random_state=56)

# fit model object
logit.fit(X_train, y_train)

# accuracy score
print(f"TF-IDF Logistic Regression model accuracy score in-sample data = {logit.score(X_train, y_train):.3}")

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="TF-IDF Logistic Regression", model_object=logit, X=X_train, y=y_train)

TF-IDF Logistic Regression model accuracy score in-sample data = 0.667


**Model #7: TF-IDF Hyperparameters**

In [45]:
# establish features and target for tfidf model

# create tfidf vectorizer object
tfidf = TfidfVectorizer()

# use tfidf object to create model features
X = tfidf.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y, train_size=.7, test_size=.3)

# create model object
tree = DecisionTreeClassifier(criterion="entropy", max_depth=5, min_samples_leaf=2,random_state=56)

# fit model object
tree.fit(X_train, y_train)

# accuracy score
print(f"TF-IDF Hyperparameters model accuracy score for in-sample data = {tree.score(X_train, y_train):.3}")

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="TF-IDF Entropy, 5, 2, Train 70%", model_object=tree, X=X_train, y=y_train)

TF-IDF Hyperparameters model accuracy score for in-sample data = 0.644


In [46]:
evaluation.sort_values(by="accuracy", ascending=False)

,model_type,accuracy
5,TF-IDF Logistic Regression,0.666667
6,"TF-IDF Entropy, 5, 2, Train 70%",0.644295
0,"CV Gini, 5",0.625731
1,"TF-IDF Gini, 5",0.602339
4,"TF-IDF Unigrams & Bigrams Gini, 5",0.590643
2,"CV Bigrams Gini, 5",0.385965
3,"TF-IDF Bigrams Gini, 5",0.385965


---
**What words are appearing most frequently depending on language?**

---

**Check out min_leaf_samples hyperparameter**

### Stratification Detour

In [47]:
# df.shape

In [48]:
# df.language.value_counts() >= 2

In [49]:
# df.groupby('language').filter(lambda x : len(x) >= 2)

In [50]:
# df.groupby("language").language.agg(["count"]).sort_values(by="count") 

In [51]:
# df[df.groupby("language").language.transform("count") >= 2]

In [52]:
# df = df[df.groupby("language").language.transform("count") >= 2]

In [53]:
# df[df.language == "ApacheConf"]